In [13]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../../../')

from notebooks.utils import _ALEXA_DATA_PATH, load_node_features, load_level_data, create_overlap_nodes, export_model_as_feature
from train import run_experiment

# Load audience overlap edges for level 2

In [14]:
level = 2
audience_overlap_sites = load_level_data(os.path.join(_ALEXA_DATA_PATH, 'corpus_2020_audience_overlap_sites_scrapping_result.json'), level=level)
audience_overlap_sites_NODES = create_overlap_nodes(audience_overlap_sites)

print(audience_overlap_sites_NODES[:5])

01-12 08:48:07 notebooks.utils INFO     Loaded 10161 nodes with records level <= 2 and child size:49342


[('crooked.com', 'votesaveamerica.com'), ('crooked.com', 'art19.com'), ('crooked.com', 'promocodeportal.com'), ('crooked.com', 'mediamatters.org'), ('crooked.com', 'actblue.com')]


In [15]:
edge_df = pd.DataFrame(audience_overlap_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,crooked.com,votesaveamerica.com
1,crooked.com,art19.com
2,crooked.com,promocodeportal.com
3,crooked.com,mediamatters.org
4,crooked.com,actblue.com


### Find all unique nodes in edges

In [16]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 26573 Sample: ['dronewars.net', 'china-doing.com', 'julie4judge.com', 'policedefense.org', 'bebatonrouge.com']


### 1. Load all node features

In [17]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [18]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [19]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26573 entries, dronewars.net to lgbtpipeline.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   17677 non-null  float64
 1   daily_pageviews_per_visitor  17684 non-null  float64
 2   daily_time_on_site           11799 non-null  float64
 3   total_sites_linking_in       25446 non-null  float64
 4   bounce_rate                  10665 non-null  float64
dtypes: float64(5)
memory usage: 1.2+ MB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [20]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26573 entries, dronewars.net to lgbtpipeline.org
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   26573 non-null  float64
 1   daily_pageviews_per_visitor  17684 non-null  float64
 2   daily_time_on_site           11799 non-null  float64
 3   total_sites_linking_in       26573 non-null  float64
 4   bounce_rate                  10665 non-null  float64
dtypes: float64(5)
memory usage: 1.2+ MB


### 3. Normalizing features

In [21]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [22]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 26573, Edges: 49372

 Node types:
  default: [26573]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [49372]
        Weights: all 1 (default)
        Features: none


# Unsupervised Attrib2Vec

In [23]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 50
epochs = 4
num_samples = [10, 5]

generator = Attri2VecLinkGenerator(G, batch_size)
train_gen = generator.flow(unsupervised_samples)

layer_sizes = [128]
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)

# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [25]:
history = model.fit(train_gen, epochs=epochs, verbose=2, use_multiprocessing=False, workers=1, shuffle=True)

Epoch 1/4
4252/4252 - 108s - loss: 0.6833 - binary_accuracy: 0.5405 - 108s/epoch - 25ms/step
Epoch 2/4
4252/4252 - 106s - loss: 0.6865 - binary_accuracy: 0.5443 - 106s/epoch - 25ms/step
Epoch 3/4
4252/4252 - 105s - loss: 0.6849 - binary_accuracy: 0.5488 - 105s/epoch - 25ms/step
Epoch 4/4
4252/4252 - 105s - loss: 0.6819 - binary_accuracy: 0.5535 - 105s/epoch - 25ms/step


In [26]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_gen = Attri2VecNodeGenerator(G, batch_size).flow(node_features_df.index.tolist())
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

print('Sample:', embeddings_wv['crooked.com'][:10], len(embeddings_wv['crooked.com']))

532/532 [==============================] - 0s 746us/step
Sample: [8.465789846923144e-09, 1.056633358587078e-08, 4.505385930997363e-08, 6.652964845699216e-09, 0.03694504499435425, 6.75667699567839e-09, 0.046012669801712036, 5.546274106649207e-09, 6.033552040207724e-07, 0.032359778881073] 128


# Export embeddings as feature

In [27]:
export_model_as_feature(embeddings_wv, f'attrib2vec_audience_overlap_level_{level}_epochs_{epochs}')

'/Users/panayot/Documents/News-Media-Peers/data/acl2020/features/attrib2vec_audience_overlap_level_2_4_epochs.json'

In [28]:
run_experiment(features=f'attrib2vec_audience_overlap_level_{level}_epochs_{epochs}')

+------+---------------------+---------------+--------------------+----------------------------------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |
+------+---------------------+---------------+--------------------+----------------------------------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_2_4_epochs |
+------+---------------------+---------------+--------------------+----------------------------------------------+


01-12 08:58:51 train        INFO     Start training...
01-12 08:58:51 train        INFO     Fold: 0
01-12 08:58:58 train        INFO     Fold: 1
01-12 08:59:01 train        INFO     Fold: 2
01-12 08:59:04 train        INFO     Fold: 3
01-12 08:59:06 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+--------------------+--------------------+--------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |      Macro-F1      |      Accuracy      |  Flip error-rate   |        MAE         |
+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+--------------------+--------------------+--------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_2_4_epochs | 33.189806467123454 | 54.249126891734576 | 15.017462165308498 | 0.6076833527357393 |
+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+--------------------+--------------------+--------------------+
